## Handle Imports

In [1]:
import numpy as np
import pandas as pd
import heapq as pq
import csv
import statistics as stats

# Upload methods

### Load Countries

In [2]:
def load_countries() -> dict:
    countries = {}

    with open("countries.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Country"]
            countries[key] = {name: float(value) for name, value in row.items() if name != "Country"}

    return countries

### Load Resources

In [3]:
def load_resources() -> dict:
    resources = {}

    with open("resources.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Resource"]
            resources[key] = {name: float(value) if name == "Weight" else value for name, value in row.items() if name != "Resource"}

    return resources

## Upload Countries

In [4]:
countryList = load_countries()

In [5]:
resourceList = load_resources()

In [6]:
print(countryList)

{'Atlantis': {'population': 20.0, 'food': 100.0, 'water': 300.0, 'land': 50.0, 'timber': 100.0, 'housing': 25.0, 'metallic element': 300.0, 'metallic alloy': 900.0, 'electronics': 400.0, 'potential fossil energy': 500.0, 'potential fossil usable': 250.0, 'community buildings': 200.0, 'jobs': 40.0, 'high school education': 20.0, 'college education': 12.0, 'universities': 4.0, 'nobel prizes': 3.0, 'marriages': 10.0, 'children': 10.0, 'renewable energy': 100.0, 'food waste': -5.0, 'water waste': -20.0, 'land waste': -20.0, 'timber waste': -50.0, 'housing waste': -5.0, 'metallicAlloy waste': -10.0, 'electronics waste': -14.0, 'potential fossil energy waste': -20.0, 'renewable energy waste': -1.0}, 'Dinotopoia': {'population': 15.0, 'food': 100.0, 'water': 100.0, 'land': 200.0, 'timber': 300.0, 'housing': 10.0, 'metallic element': 400.0, 'metallic alloy': 100.0, 'electronics': 200.0, 'potential fossil energy': 1000.0, 'potential fossil usable': 100.0, 'community buildings': 20.0, 'jobs': 30

In [7]:
print(resourceList)

{'R1': {'Weight': 0.0, 'Notes': 'analog to population'}, 'R2': {'Weight': 0.0, 'Notes': 'analog to metalic elements'}, 'R3': {'Weight': 0.0, 'Notes': 'analog to timber'}, 'R4': {'Weight': 0.2, 'Notes': 'analog to metallic alloys'}, 'R5': {'Weight': 0.5, 'Notes': 'analog to electronics'}, 'R6': {'Weight': 0.8, 'Notes': 'analog to housing'}, 'R7': {'Weight': -0.5, 'Notes': 'waste'}, 'R8': {'Weight': -0.8, 'Notes': 'waste'}, 'R9': {'Weight': -0.4, 'Notes': 'waste'}, 'R10': {'Weight': -0.2, 'Notes': 'waste'}}


### Set to Pandas Dataframe

In [8]:
country_df = pd.DataFrame(countryList).transpose()

In [9]:
resource_df = pd.DataFrame(resourceList).transpose()

In [10]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,100.0,300.0,50.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,250.0,20.0,100.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


In [11]:
resource_df

,Weight,Notes
R1,0.0,analog to population
R2,0.0,analog to metalic elements
R3,0.0,analog to timber
R4,0.2,analog to metallic alloys
R5,0.5,analog to electronics
R6,0.8,analog to housing
R7,-0.5,waste
R8,-0.8,waste
R9,-0.4,waste
R10,-0.2,waste


# State Quality Function

### Check Country List

In [12]:
for key in countryList:
    print(key, '->', countryList[key])

Atlantis -> {'population': 20.0, 'food': 100.0, 'water': 300.0, 'land': 50.0, 'timber': 100.0, 'housing': 25.0, 'metallic element': 300.0, 'metallic alloy': 900.0, 'electronics': 400.0, 'potential fossil energy': 500.0, 'potential fossil usable': 250.0, 'community buildings': 200.0, 'jobs': 40.0, 'high school education': 20.0, 'college education': 12.0, 'universities': 4.0, 'nobel prizes': 3.0, 'marriages': 10.0, 'children': 10.0, 'renewable energy': 100.0, 'food waste': -5.0, 'water waste': -20.0, 'land waste': -20.0, 'timber waste': -50.0, 'housing waste': -5.0, 'metallicAlloy waste': -10.0, 'electronics waste': -14.0, 'potential fossil energy waste': -20.0, 'renewable energy waste': -1.0}
Dinotopoia -> {'population': 15.0, 'food': 100.0, 'water': 100.0, 'land': 200.0, 'timber': 300.0, 'housing': 10.0, 'metallic element': 400.0, 'metallic alloy': 100.0, 'electronics': 200.0, 'potential fossil energy': 1000.0, 'potential fossil usable': 100.0, 'community buildings': 20.0, 'jobs': 30.0

### Set Maslow Constants

In [13]:
#different resource levels
levelOneResources = {'food': 1, 'water': 1}
levelTwoResources = {'housing': 1, 'timber': 1, 'metallic alloy': 0.5, 'electronics': 3, 'potential fossil energy': 1} 
levelThreeResources = {'community buildings': 0.05, 'jobs': 1, 'high school education': 1, 'college education': 1, 'universities': 1, 'marriages': 1} 
levelFourResources = {'children': 2.5, 'renewable energy': 1}
levelFiveResources = {'food waste': -1, 'water waste': -1, 'land waste': -1, 'timber waste': -1, 'nobel prizes': 0.02}

#list of levels
levList = [levelOneResources, levelTwoResources, levelThreeResources, levelFourResources, levelFiveResources]

In [14]:
print(levList)

[{'food': 1, 'water': 1}, {'housing': 1, 'timber': 1, 'metallic alloy': 0.5, 'electronics': 3, 'potential fossil energy': 1}, {'community buildings': 0.05, 'jobs': 1, 'high school education': 1, 'college education': 1, 'universities': 1, 'marriages': 1}, {'children': 2.5, 'renewable energy': 1}, {'food waste': -1, 'water waste': -1, 'land waste': -1, 'timber waste': -1, 'nobel prizes': 0.02}]


### Level Function

In [15]:
#pandas version
def leveldf(df, country, level, resources):
    
    levelSat = True
    mult = []
    average = 0

    for key, value in levList[level-1].items():
        countryVal = df.loc[country, key]
        mult.append(countryVal/value)
        if countryVal < value:
            levelSat = False
    
    average = stats.mean(mult)
    
    if not levelSat:
        average = average/4
        
    return average

In [16]:
num = leveldf(country_df, 'Atlantis', 1, levList)
num

200.0

## Maslow Function

In [17]:
def maslow(df, country, level):
    
    maslowList = []
    
    ## normalize
    norm_df = df.copy()
    
    for row in range(len(norm_df)):
        
        values = norm_df.iloc[row]
        popVal = values[0]
        
        for vals in range(1, len(values)):
            values[vals] = values[vals]/popVal
    
    # maslow function
    for num in range(1, level+1):
        levValue = leveldf(norm_df, country, num, levList)
        maslowList.append(levValue)
            
    return maslowList

In [18]:
maslowL = maslow(country_df, 'The Vale', 5, levList)

In [19]:
print(maslowL)

[0.7848837209302325, 2.357364341085271, 4.944767441860465, 0.2558139534883721, 0.4127906976744186]


In [20]:
def maslowHeuristicVal(lst):
    return stats.mean(maslowL)

In [21]:
masVal = maslowHeuristicVal(maslowL)

In [22]:
print(masVal)

1.7511240310077518


# Transform Templates

In [23]:
HOUSING = [{'land': 1, 'population': 5, 'water': 5, 'metallic element': 1, 'timber': 5, 'metallic alloy': 3, 'potential fossil usable': 5}, {'housing': 1, 'housing waste': 1, 'timber waste': 1, 'population': 5, 'water': 4}]
ALLOYS = [{'population': 1, 'metallic element': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'metallic alloy': 1, 'metallicAlloy waste': 1, 'water': 2}]
ELECTRONICS = [{'population': 1, 'metallic element': 3, 'metallic alloy': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'electronics': 2, 'electronics waste': 1, 'water': 2}]
FARM = [{'population': 1, 'land' : 1, 'water': 3}, {'food': 5, 'population': 1}]
LOGGING = [{'population': 3, 'potential fossil usable': 3}, {'population': 3, 'timber': 5}]
PURIFY_WATER = [{'population': 3, 'potential fossil usable': 3}, {'population': 3, 'water': 5}]
FOSSIL_ENERGY = [{'population': 5, 'potential fossil energy': 2}, {'population': 5, 'potential fossil usable': 1, 'potential fossil energy waste': 1}]
RENEWABLE_ENERGY = [{'population': 5, 'potential fossil usable': 3}, {'population': 5, 'renewable energy': 1, 'renewable energy waste': 1}]
COMMUNITY_BUILDING = [{'land': 1, 'population': 10, 'water': 5, 'metallic element': 3, 'timber': 8, 'metallic alloy': 5, 'potential fossil usable': 5}, {'community buildings': 1, 'housing waste': 1, 'timber waste': 1, 'metallicAlloy waste': 1, 'population': 10, 'water': 4}]
UNIVERSITY = [{'land': 1, 'population': 50, 'water': 5, 'metallic element': 5, 'timber': 10, 'metallic alloy': 5, 'potential fossil usable': 5}, {'universities': 1, 'population': 50, 'water': 3, 'timber waste': 1, 'metallicAlloy waste': 1}]
JOB_HS = [{'population': 25, 'high school education': 1}, {'population': 25, 'jobs': 1}]
JOB_C = [{'population': 50, 'college education': 1}, {'population': 50, 'jobs': 1}]
HIGHSCHOOL_ED = [{'population': 15, 'housing': 1, 'children': 1}, {'population': 16, 'housing': 1, 'high school education': 1}]
COLLEGE_ED = [{'population': 50, 'housing': 1, 'universities': 1, 'high school education': 1}, {'population': 50, 'housing': 1, 'universities': 1, 'college education': 1}]
MARRIAGE = [{'population': 2, 'housing': 1}, {'population': 2, 'housing': 1, 'marriages': 1}]
CHILDREN = [{'marriages': 1, 'housing': 1}, {'marriages': 1, 'housing': 1, 'children': 2}]
NOBEL_PRIZE = [{'population': 1, 'universities': 10, 'college education': 50, 'potential fossil usable': 10}, {'population': 1, 'universities': 10, 'college education': 50, 'nobel prizes': 1}]

# Transform Function

In [24]:
def transform(df, country, transform):
    
    allowed = True
    
    #check if transform is possible
    for key in transform[0]:
        val = transform[0][key]
        if(df.loc[country, key] - val < 0):
            allowed = False
    
    if(allowed):
        #remove input resoures
        for key in transform[0]:
            val = transform[0][key]
            df.loc[country, key] -= val

        #add output resources
        for key in transform[1]:
            val = transform[1][key]
            df.loc[country, key] += val

# Transform Test Case

In [25]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,100.0,300.0,50.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,250.0,20.0,100.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


In [26]:
transform(country_df, 'Atlantis', FARM)

In [27]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,105.0,297.0,49.0,100.0,25.0,300.0,900.0,400.0,500.0,...,100.0,-5.0,-20.0,-20.0,-50.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,100.0,100.0,200.0,300.0,10.0,400.0,100.0,200.0,1000.0,...,200.0,-2.0,-400.0,-80.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,300.0,300.0,400.0,100.0,100.0,30.0,2600.0,100.0,2000.0,...,100.0,-10.0,-30.0,-20.0,-100.0,-120.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,300.0,300.0,200.0,200.0,100.0,100.0,200.0,2000.0,...,150.0,-3.0,-200.0,-10.0,-20.0,-23.0,-98.0,-100.0,-25.0,-20.0
The Vale,43.0,250.0,20.0,100.0,54.0,300.0,100.0,320.0,100.0,1000.0,...,80.0,-10.0,-100.0,-50.0,-45.0,-50.0,-10.0,-20.0,-100.0,-18.0


# Transfer Function Definition

In [28]:
def transfer(df, country1, country2, transfer):

    allowed = True

    #check for validityy

    for key in transfer:
        val = transfer[key]
        if (df.loc[country1,key] - val < 0):
            allowed = False

    if(allowed):
        #remove resource from country 1
        for key in transfer:
            val = transfer[key]
            df.loc[country1,key] -= val

        #add resource to country 2
        for key in transfer:
            val = transfer[key]
            df.loc[country2,key] += val



# Transfer Templates

In [37]:
FOOD = {"food":5}
WATER = {"water":5}
TIMBER = {"timber":5}
METALLIC_ELEMENT = {"metallic element":5}
METALLIC_ALLOY = {"metallic alloy":5}
ELECTRONICS_TRANSFER = {"electronics":5}
POTENTIAL_FOSSIL_ENERGY = {"potential fossil energy":5}
POTENTIAL_FOSSIL_USABLE = {"potential fossil usable":5}
RENEWABLE_ENERGY_TRANSFER = {"renewable energy":5}

ALL_TEMPLATES_TRANSFER = {'FOOD','WATER','TIMBER','METALLIC_ELEMENT','METALLIC_ALLOY','ELECTRONICS_TRANSFER',
                          'POTENTIAL_FOSSIL_ENERGY','POTENTIAL_FOSSIL_USABLE','RENEWABLE_ENERGY_TRANSFER'}

# Trade Function Definition

In [42]:
def trade(df, country1, country2, transfer1, transfer2) -> bool:
    # A trade deal between two countries country1 and country2
    # for which country uses transfer1 and country2 uses transfer2
    if transfer1 == transfer2:
        return False
    if transfer(df, country1, country2, transfer1):
        if not transfer(df, country2, country1, transfer2):
            transfer(df, country2, country1, transfer1)
            return False
        else:
            return True
    else:
        return False

# Node Definition

In [43]:
class Node:
    def __init__(self, state_quality, exp_util, sched, countrydf):
        self.state_quality = state_quality
        self.exp_util = exp_util
        self.sched = sched
        self.countrydf = countrydf
    def getSqual(self):
        return self.squal
    def getExutil(self):
        return self.exutil
    def getSched(self):
        return self.sched
    def getCountrydf(self):
        return self.countrydf

# Bounded Priority Queue Definition

In [44]:
class BoundedPriorityQueue:
    def __init__(self, bound=None):
        self.bound = bound
        self.queue = list()
    def push(self, priority, item):
        if len(self.queue) == self.bound:
            pq.heappushpop(self.queue, (priority, item))
        else:
            pq.heappush(self.queue, (priority, item))
    def pop(self):
        return pq.heapop(self.queue)[-1]
    def clear(self):
        self.queue.clear()


## Generate Succcessors

In [45]:
def generate_successors(parentNode, my_country):
    
    successors = BoundedPriorityQueue()
    prevSched = parentNode.getSched()
    
    for i in ALL_TEMPLATES_TRANSFORM:
        
        new_df = parentNode.getCountrydf()
        
        if transform(new_df, my_country,i):
            state_qual = maslowHeuristicVal(maslow(new_df, my_country, 5))
            new_state = Node(state_qual, 0, prevSched, new_df)
            new_state.sched.append(i)
            #to do expUtility
            new_state.exp_util = 0
            successors.push(new_state.exp_util, new_state)


    for j in ALL_TEMPLATES_TRANSFER:
        for k in ALL_TEMPLATES_TRANSFER:
            for l in countryList:
                
                new_df = parentNode.getCountrydf()
                
                if trade(new_df,my,l,j,k):
                    new_state = Node(sq, eu, action, new_df)
                    successors.push(eu,new_state)